# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy
#jupyter nbextension enable --py gmaps

# Import API key
from api_keys import g_key
# Import API key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# OpenWeatherMap API Key
api_key = g_key

In [10]:
api_key = weather_api_key

In [14]:
# Store filepath in a variable
# WeatherPyCities = "WeatherPy_data.csv"
#  # Read our Data file with the pandas library, including encoding
# WeatherPyCities = pd.read_csv("../Output_data/WeatherPy_data.csv, encoding="ISO-8859-1")
# WeatherPyCities.head()
WeatherPyCities = pd.read_csv("../WeatherPy/output_data/WeatherPy_data.csv", encoding="ISO-8859-1")
WeatherPyCities.head()       

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,0,Esperance,90,AU,1612469312,76,-33.8667,121.9000,55.40,13.13
1,1,Rikitea,82,PF,1612469312,68,-23.1203,-134.9692,78.46,10.09
2,2,Cabinda,100,AO,1612469313,85,-5.5500,12.2000,80.74,7.09
3,3,Te Anau,65,NZ,1612469313,79,-45.4167,167.7167,55.13,1.12
4,4,Bluff,49,NZ,1612469313,77,-46.6000,168.3333,48.79,6.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# create variables
locations = WeatherPyCities[["Lat", "Longitude"]]
weight = WeatherPyCities["Humidity"]

In [16]:
# create custom Heatmap - customized size
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

In [20]:
# Plot Heatmap
fig = gmaps.figure(center=(40.0, -5.0), zoom_level=2)
max_intensity = np.max(weight)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weight, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
holiday_cities = WeatherPyCities[(WeatherPyCities["Cloudiness"]<30) & (WeatherPyCities["Humidity"]<30) &
 (WeatherPyCities["Max Temp"]>70) & (WeatherPyCities["Max Temp"]<85) & (WeatherPyCities["Wind Speed"]<10)]
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
23,23,Markala,25,ML,1612469320,14,13.7021,-6.0659,79.75,6.87
243,243,Diapaga,0,BF,1612469171,16,12.0708,1.7889,80.89,7.81
278,278,Ojinaga,1,MX,1612469372,22,29.5667,-104.4167,75.20,5.75
291,291,Tecpan,20,GT,1612469374,22,14.7623,-90.9947,80.60,3.44
295,295,GourÃ©,0,NE,1612469375,18,13.9835,10.2704,73.00,5.01
355,355,Dutse,0,NG,1612469386,20,11.7594,9.3392,73.78,9.60
380,380,Prieska,0,ZA,1612469391,14,-29.6641,22.7474,84.33,5.30
461,461,KyabÃ©,1,TD,1612469406,16,9.4515,18.9449,83.19,6.82
524,524,Dogondoutchi,0,NE,1612469418,17,13.6393,4.0287,78.40,9.84


In [22]:
holiday_cities.reset_index(drop=True, inplace=True)
holiday_cities

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,23,Markala,25,ML,1612469320,14,13.7021,-6.0659,79.75,6.87
1,243,Diapaga,0,BF,1612469171,16,12.0708,1.7889,80.89,7.81
2,278,Ojinaga,1,MX,1612469372,22,29.5667,-104.4167,75.20,5.75
3,291,Tecpan,20,GT,1612469374,22,14.7623,-90.9947,80.60,3.44
4,295,GourÃ©,0,NE,1612469375,18,13.9835,10.2704,73.00,5.01
5,355,Dutse,0,NG,1612469386,20,11.7594,9.3392,73.78,9.60
6,380,Prieska,0,ZA,1612469391,14,-29.6641,22.7474,84.33,5.30
7,461,KyabÃ©,1,TD,1612469406,16,9.4515,18.9449,83.19,6.82
8,524,Dogondoutchi,0,NE,1612469418,17,13.6393,4.0287,78.40,9.84


In [23]:
holiday_cities = holiday_cities.drop('Unnamed: 0', axis=1)
holiday_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed
0,Markala,25,ML,1612469320,14,13.7021,-6.0659,79.75,6.87
1,Diapaga,0,BF,1612469171,16,12.0708,1.7889,80.89,7.81
2,Ojinaga,1,MX,1612469372,22,29.5667,-104.4167,75.20,5.75
3,Tecpan,20,GT,1612469374,22,14.7623,-90.9947,80.60,3.44
4,GourÃ©,0,NE,1612469375,18,13.9835,10.2704,73.00,5.01
5,Dutse,0,NG,1612469386,20,11.7594,9.3392,73.78,9.60
6,Prieska,0,ZA,1612469391,14,-29.6641,22.7474,84.33,5.30
7,KyabÃ©,1,TD,1612469406,16,9.4515,18.9449,83.19,6.82
8,Dogondoutchi,0,NE,1612469418,17,13.6393,4.0287,78.40,9.84


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# add new column for hotel name
hotel_df = holiday_cities
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Longitude,Max Temp,Wind Speed,Hotel Name
0,Markala,25,ML,1612469320,14,13.7021,-6.0659,79.75,6.87,
1,Diapaga,0,BF,1612469171,16,12.0708,1.7889,80.89,7.81,
2,Ojinaga,1,MX,1612469372,22,29.5667,-104.4167,75.20,5.75,
3,Tecpan,20,GT,1612469374,22,14.7623,-90.9947,80.60,3.44,
4,GourÃ©,0,NE,1612469375,18,13.9835,10.2704,73.00,5.01,


In [25]:
# begin looping through hotel dataframe
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {"radius": 5000,
                  "types": "lodging",
                   "keyword": "Hotel",
                   "key": g_key}

        lat = row['Lat']
        long = row['Longitude']
        city = row['City']
        params['location'] = f"{lat}, {long}"

        hotel_name = requests.get(base_url, params=params).json()
        results = hotel_name['results']
        
        print(f"First hotel in {city} is {results[0]['name']}.") 
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except IndexError:
        print("No hotel found.")

No hotel found.
No hotel found.
First hotel in Ojinaga is Riata Inn.
First hotel in Tecpan is Raqui Lodge.
No hotel found.
First hotel in Dutse is Bege Guest Inn.
First hotel in Prieska is Excelsior River Cottage.
No hotel found.
First hotel in Dogondoutchi is Hôtel Alheri Maison Alheri.


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Longitude"]]

In [27]:
# create custom Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…